In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://i.pinimg.com/originals/0c/ab/1c/0cab1c6c171b4e0d71ec822600f5923b.jpg)

# Predicting the Price of Electricty with Machine Learning

"Real-time electricity pricing models can potentially lead to economic and environmental advantages compared to the current common flat rates. In particular, they can provide end users with the opportunity to reduce their electricity expenditures by responding to pricing that varies with different times of the day. However, recent studies have revealed that the lack of knowledge among users about how to respond to time-varying prices as well as the lack of effective building automation systems are two major barriers for fully utilizing the potential benefits of real-time pricing tariffs. We tackle these problems by proposing an optimal and automatic residential energy consumption scheduling framework which attempts to achieve a desired trade-off between minimizing the electricity payment and minimizing the waiting time for the operation of each appliance in household in presence of a real-time pricing tariff combined with inclining block rates . Our design requires minimum effort from the users and is based on simple linear programming computations. Moreover, we argue that any residential load control strategy in real-time electricity pricing environments requires price prediction capabilities. This is particularly true if the utility companies provide price information only one or two hours ahead of time. Simulation results show that the combination of the proposed energy consumption scheduling design and the price predictor filter leads to significant reduction not only in users' payments but also in the resulting peak-to-average ratio in load demand for various load scenarios. Therefore, the deployment of the proposed optimal energy consumption scheduling schemes is beneficial for both end users and utility companies." -[Amir-Hamed Mohsenian-Rad](https://ieeexplore.ieee.org/abstract/document/5540263)




## Data Contents -

This dataset contains four years of electrical consumption, generation, pricing, and weather data for Spain. Consumption and generation data was retrieved from ENTSOE a public portal for Transmission Service Operator (TSO) data. Settlement prices were obtained from the Spanish TSO Red Electric España. 

## Column Meanings - 

* Time: Datetime index localized to CET
* Generation biomass: biomass generation in MW
* Generation fossil brown coal/lignite: coal/lignite generation in MW
* Generation fossil coal-derived gas: coal gas generation in MW
* Generation fossil gas: gas generation in MW
* Generation fossil hard coal: coal generation in MW
* Generation fossil oil: oil generation in MW
* Generation fossil oil shale: shale oil generation in MW
* Generation fossil peat: peat generation in MW
* Generation geothermal: geothermal generation in MW
* Generation hydro pumped storage aggregated: hydro1 generation in MW
* Generation hydro pumped storage consumption: hydro2 generation in MW
* Generation hydro run-of-river and poundage: hydro3 generation in MW
* Generation hydro water reservoir: hydro4 generation in MW
* Generation marine: sea generation in MW
* Generation nuclear: nuclear generation in MW
* Generation other: other generation in MW
* Generation other renewable: other renewable generation in MW
* Generation solar: solar generation in MW
* Generation waste: waste generation in MW
* Generation wind offshore: wind offshore generation in MW
* Generation wind onshore: wind onshore generation in MW
* Gorecast solar day ahead: forecasted solar generation
* Gorecast wind offshore eday ahead: forecasted offshore wind generation
* Gorecast wind onshore day ahead: forecasted onshore wind generation
* Total load forecast: forecasted electrical demand
* Total load actual: actual electrical demand
* Price day ahead: forecasted price EUR/MWh
* Price actual: price in EUR/MWh

In [ ]:
# Import libraries -

# Visualization
import shap
%matplotlib inline
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import eli5
from eli5.sklearn import PermutationImportance

# Data Wrangling
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',50)

# Model Creation
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.linear_model import Ridge, LinearRegression
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve, GridSearchCV
from sklearn.metrics import roc_curve, plot_roc_curve, mean_absolute_error, mean_squared_error, accuracy_score

# Warnings Ignore
import warnings
warnings.filterwarnings("ignore")

# 1. Data Exploration / Clean - 



In [ ]:
# Read in data
df = pd.read_csv('../input/energy-consumption-generation-prices-and-weather/energy_dataset.csv')

# Preview of data
df.head(5)

## 1.1. Data Exploration -

In [ ]:
# .Describe to show overview of data
df.describe()

In [ ]:
# .Info to show datatype, nulls, and count
df.info()

In [ ]:
# Null % check amongst columns
round((df.isnull().sum()/len(df)*100),2)

In [ ]:
# DF.corr to show correlation of values
df.corr()

In [ ]:
# Correlation of columns to target variable 
correlations = df.corr(method='pearson')
print(correlations['price actual'].sort_values(ascending=False).to_string())

In [ ]:
# Assign Variable to drop columns
zero_val_cols = ['generation marine',
                 'generation geothermal',
                 'generation fossil peat',
                 'generation wind offshore',
                 'generation fossil oil shale',
                 'forecast wind offshore eday ahead',
                 'generation fossil coal-derived gas',
                 'generation hydro pumped storage aggregated']

# Drop Columns with zero values
heat_map_features = df.drop(columns=zero_val_cols,axis=1)

# Set Figure Size
plt.figure(figsize=(15,12.5))

# .corr heatmap of df to visualize correlation & show plot
sns.heatmap(round(heat_map_features.corr(),1),annot=True,cmap='Blues',linewidth=0.9)
plt.show();

In [ ]:
# Figure Size
plt.figure(figsize=(15,10))

# Hist graph to show distribution of target variable
sns.histplot(df,x='price actual');

## 1.2. Data Exploration Findings -
* Normal distrubution of target variable.
* Low percentage of nan values in columns without 100% nan.
* Lots of columns with 0 values for certain energy genreation types.
* Found very high correlation between some columns.
* All data seems to be numeric.

## 1.3. Data Cleaning -

In [ ]:
# Wrangle function to clean data
def wrangle(filepath):
    
    '''
                 ,,,,,,,
      (\-"""-/) /       \
       |     | /         \
       \ 0 0 //           \
        \_o_//       /\   /
       /`   `\      |  \,/  
      /       \     |  
      \ (   ) /     |
     / \_)-(_/ \    |
    |  /_____\  |  /
    \  \ N.C / /  /
     \ '.___.' / /
    .'  \-=-/  '.
   /   /`   `\   \
  (//./       \.\\)
   `"`         `"`
   
    '''
    
    # Read in the data, parse dates, and set the index
    df = pd.read_csv(filepath,parse_dates=['time'],index_col='time')
 
    # Rename columns by replacing all - or blank space with _
    df.columns = df.columns.str.replace(' ','_').str.replace('-','_')

    # Make the index DT
    df.index = pd.to_datetime(df.index, utc=True)    

    # Drop all columns with data leakage, or 90% + null
    df.drop(columns=['price_day_ahead',
                     'generation_marine',
                     'total_load_forecast',
                     'generation_geothermal',
                     'generation_fossil_peat',
                     'generation_wind_offshore',
                     'forecast_solar_day_ahead',
                     'generation_fossil_oil_shale',
                     'forecast_wind_onshore_day_ahead',
                     'forecast_wind_offshore_eday_ahead',
                     'generation_fossil_coal_derived_gas',
                     'generation_hydro_pumped_storage_aggregated'],inplace=True)
    
    # Drop Outlier row 2014 for plotting
    df = df.drop(pd.Timestamp('2014-12-31 23:00:00+00:00')) 
    
    # Sort index
    df = df.sort_index()
    
    # Set conditional satements for filtering times of month to season value
    condition_winter = (df.index.month>=1)&(df.index.month<=3)
    condtion_spring = (df.index.month>=4)&(df.index.month<=6)
    condition_summer = (df.index.month>=7)&(df.index.month<=9)
    condition_automn = (df.index.month>=10)@(df.index.month<=12)
    
    # Create column in dataframe that inputs the season based on the conditions created above
    df['season'] = np.where(condition_winter,'winter',
                            np.where(condtion_spring,'spring',
                                     np.where(condition_summer,'summer',
                                              np.where(condition_automn,'automn',np.nan))))

    return df

# Applying the wrangle function to the dataset
df=wrangle('../input/energy-consumption-generation-prices-and-weather/energy_dataset.csv')

## 1.4. Data Cleaning Notes -
* Removed forecasted columns to prevent data leakage.
* Removed all columns with 0 fill for all values.
* Removed only row from 2014 for plotting purposes.
* Feature engineered seasons using time of year.

In [ ]:
# Figure showing Price per total load
fig = px.scatter(df,x='total_load_actual',
                 y='price_actual',
                 facet_col='season',
                 opacity=0.1,
                 title='Price Per KW Hour Compaired To Total Energy Genereated Per Season',
                 animation_frame=df.index.year)

# Figure customizations
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='darkslateblue')),
                  selector=dict(mode='markers'))

# 2. ML Model Creation -

## 2.1. Split Data -

In [ ]:
# Create Target variable
target='price_actual'

# Split data into feature matrix and target vector
y,X=df[target],df.drop(columns=target)

# split data into train / validation sets
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=.2,random_state=42)

## 2.2. Baselines -

In [ ]:
# Assign variables for baselines and calculate baselines
y_pred = [y_train.mean()]*len(y_train)
mean_baseline_pred = y_train.mean()
baseline_mae = mean_absolute_error(y_train,y_pred)
baseline_rmse = mean_squared_error(y_train,y_pred,squared=False)

# Print statement to show all baseline values
print('Mean Price Per KW/h Baseline Pred:', mean_baseline_pred)
print('-------------------------------------------------------------------')
print('Baseline Mae:',baseline_mae)
print('-------------------------------------------------------------------')
print('Baseline RMSE:',baseline_rmse)

## 2.3. Default Model Creation -

In [ ]:
# One Hot Encoder to transform Seasons column
onehot = OneHotEncoder(use_cat_names=True)
onehot_fit = onehot.fit(X_train)
XT_train = onehot.transform(X_train)
XT_val = onehot.transform(X_val)

# Simple imputer to fill nan values, then transform sets
simp = SimpleImputer(strategy='mean')
simp_fit = simp.fit(XT_train)
XT_train = simp.transform(XT_train)
XT_val = simp.transform(XT_val)

# Assigning model variables
model_lr=LinearRegression()
model_r=Ridge()

# Fitting models
model_r.fit(XT_train,y_train);
model_lr.fit(XT_train,y_train);

# Def to check model metrics of baseline performance
def check_metrics(model):
    print(model)
    print('===================================================================')
    print('Training MAE:', mean_absolute_error(y_train,model.predict(XT_train)))
    print('-------------------------------------------------------------------')
    print('Validation MAE:', mean_absolute_error(y_val,model.predict(XT_val)))
    print('-------------------------------------------------------------------')
    print('Validation R2 score:', model.score(XT_val,y_val))
    print('===================================================================')
model = [model_r,model_lr]
for m in model:
  check_metrics(m)

In [ ]:
# Ordinal Encoder to transform Seasons column
ordinal = OrdinalEncoder()
ordinal_fit = ordinal.fit(X_train)
XT_train = ordinal.transform(X_train)
XT_val = ordinal.transform(X_val)

# Simple imputer to fill nan values, then transform sets
simp = SimpleImputer(strategy='mean')
simp_fit = simp.fit(XT_train)
XT_train = simp.transform(XT_train)
XT_val = simp.transform(XT_val)

# Assigning model variables
model_rfr = RandomForestRegressor()
model_xgbr=XGBRegressor()

# Fitting models
model_rfr.fit(XT_train,y_train);
model_xgbr.fit(XT_train,y_train);

# Def to check model metrics of baseline performance
def check_metrics(model):
    print(model)
    print('===================================================================')
    print('Training MAE:', mean_absolute_error(y_train,model.predict(XT_train)))
    print('-------------------------------------------------------------------')
    print('Validation MAE:', mean_absolute_error(y_val,model.predict(XT_val)))
    print('-------------------------------------------------------------------')
    print('Validation R2 score:', model.score(XT_val,y_val))
    print('===================================================================')
model = [model_xgbr,model_rfr]
for m in model:
  check_metrics(m)

## 2.3.1. Default Model Results (Without Feature Engineering) -

#### Ridge -
* Training MAE: 8.472823575635683
* Validation MAE: 8.63383328893237
* R2 score: 0.35638687883377895

#### LinearRegression - 
* Training MAE: 8.472823557244569
* Validation MAE: 8.633833278291645
* R2 score: 0.3563868756924954

#### XGBRegressor -
*  Training MAE: 3.545327560560801
*  Validation MAE: 4.6979849457995755
*  R2 score: 0.7950966649998653

#### RandomForestRegressor -
*  Training MAE: 1.4598095162420663
*  Validation MAE: 3.9252298502111427
*  R2 score: 0.8349323474716535

### 2.3.2. Defualt Model Observations -

* RandomForestRegressor seems to out perform any other model.
* Ridge Regression / Linear Regression have very similar outputs with no parameter tuning.

### 2.3.3. Default Model + Feature Engineering Observations -

* Increase of validation scores from 3-7%
    * HUGE improvement

## 2.4. XGB RandomSearch -

In [ ]:
# Pipeline variable for RandomSearch
pipe_rs_xgb = make_pipeline(OrdinalEncoder(),
                            SimpleImputer(),
                            XGBRegressor(random_state=42,
                                         n_jobs=-1))

# Params for RandomSearch
paramajama = {'simpleimputer__strategy':['meadian','mean'],
              'xgbregressor__max_depth':range(5,35,5),
              'xgbregressor__learning_rate':np.arange(0.2,1,0.1),
              'xgbregressor__booster':['gbtree','gblinear','dart'],
              'xgbregressor__min_child_weight':range(1,10,1),
              'xgbregressor__gamma':np.arange(0,1,0.1),
              'xgbregressor__max_delta_step':np.arange(0,1,0.1),
              'xgbregressor__subsample':np.arange(0.5,1,0.1)}

# RandomSearch Model
model_rs_xgbr = RandomizedSearchCV(pipe_rs_xgb,
                                   param_distributions = paramajama,
                                   n_iter=20,
                                   n_jobs=-1)

# Model fit
model_rs_xgbr.fit(X_train,y_train);

# Check model metrics
print('Training MAE:', mean_absolute_error(y_train,model_rs_xgbr.predict(X_train)))
print('-------------------------------------------------------------------')
print('Validation MAE:', mean_absolute_error(y_val,model_rs_xgbr.predict(X_val)))
print('-------------------------------------------------------------------')
print('R2 score:', model_rs_xgbr.score(X_val,y_val))
print('===================================================================')
model_rs_xgbr.best_params_

## 2.4.1. XGB RandomSearch Conclusions -

(20 random searches with 100 iter. Scaled down for kaggle notebook import) 
* RandomSearch model out performed the default with tuning.
* Highest score listed - .8991
    * Highest scoring model.
* I'm interested in grid search performance post project to see if the model can outperform the random searches.

### Best Score -
* Training MAE: 0.18182096506754555
* Validation MAE: 3.143150404572878
* R2 score: 0.8991961935947261

#### Params -
*  'xgbregressor__subsample': 0.7,
*  'xgbregressor__min_child_weight': 4,
*  'xgbregressor__max_depth': 25,
*  'xgbregressor__max_delta_step': 0.0,
*  'xgbregressor__learning_rate': 0.2,
*  'xgbregressor__gamma': 0.5,
*  'xgbregressor__booster': 'dart',
*  'simpleimputer__strategy': 'mean'}
 
### Second Best Score -
* Training MAE: 0.22435663695989874
* Validation MAE: 3.6606957501289457
* R2 score: 0.8619067640247029

## 2.5. Random Forest Regressor RandomSearch -

In [ ]:
# Pipeline variable for RandomSearch
pipe_rs_rfr = make_pipeline(OrdinalEncoder(),
                            SimpleImputer(),
                            RandomForestRegressor(random_state=42,
                                                  n_jobs=-1))

# Params for RandomSearch
pramajams = {'simpleimputer__strategy':['mean','meadian'],
             'randomforestregressor__max_depth':range(5,35,5),
             'randomforestregressor__n_estimators':range(25,200,10),
             'randomforestregressor__max_samples':np.arange(0.2,1,0.1),
             'randomforestregressor__max_features':['sqrt','log2'],
             'randomforestregressor__min_samples_split':np.arange(2,5,1)}

# RandomSearch Model
model_rs_rfr = RandomizedSearchCV(pipe_rs_rfr,
                                  param_distributions=pramajams,
                                  n_iter=20,
                                  n_jobs=-1)

# Model fit
model_rs_rfr.fit(X_train,y_train);

# Check model metrics
print('Training MAE:', mean_absolute_error(y_train,model_rs_rfr.predict(X_train)))
print('-------------------------------------------------------------------')
print('Validation MAE:', mean_absolute_error(y_val,model_rs_rfr.predict(X_val)))
print('-------------------------------------------------------------------')
print('R2 score:', model_rs_rfr.score(X_val,y_val))
print('===================================================================')
model_rs_rfr.best_params_

## 2.5.1. Random Forest Regressor RandomSearch Conclusions -

(20 random searches with 100 iter. Scaled down for kaggle notebook import) 
* Default model out performed the RandomSearch with tuning.
* Highest score listed - .8795
* Model did not out perform the default model.

### Best Score -

* Training MAE: 1.4004700781578163
* Validation MAE: 3.371529047081576
* R2 score: 0.8795676540229065

#### Params -
*  'simpleimputer__strategy': 'mean',
*  'randomforestregressor__n_estimators': 115,
*  'randomforestregressor__min_samples_split': 2,
*  'randomforestregressor__max_samples': 0.9000000000000001,
*  'randomforestregressor__max_features': 'sqrt',
*  'randomforestregressor__max_depth': 25
 
### Second Best Score -

* Training MAE: 1.4371681333507431
* Validation MAE: 3.398423017100202
* R2 score: 0.877617516249465

## 2.6. Grid Search RandomForestRegressor -

In [ ]:
# Pipeline variable for GridSearch
# pipe_it_up = make_pipeline(OrdinalEncoder(),
#                            SimpleImputer(),
#                            RandomForestRegressor(random_state=42,n_jobs=-1))

# Params for GridSearch
# pramajams = {'randomforestregressor__max_depth':range(0,35,5),
#              'randomforestregressor__n_estimators':range(100,200,10),
#              'randomforestregressor__max_samples':np.arange(0,1,0.1),
#              'randomforestregressor__max_features':['sqrt','log2'],
#              'randomforestregressor__criterion':['mse','mae'],
#              'randomforestregressor__min_samples_split':np.arange(2,5,1)}

# GridSearch Model
# model_rfr_two = GridSearchCV(pipe_it_up,
#                              param_grid=pramajams,
#                              n_jobs=-1)

# Model fit
# model_rfr_two.fit(X_train,y_train)

# Check model metrics
# print('Training MAE:',mean_absolute_error(y_train,model_rfr_two.predict(X_train)))
# print('-------------------------------------------------------------------')
# print('Validation MAE:',mean_absolute_error(y_val,model_rfr_two.predict(X_val)))
# print('-------------------------------------------------------------------')
# print('R2 score:',model_rfr_two.sore(X_val,y_val))
# print('===================================================================')
# model_rfr_two.best_params_

## 2.7. Results -

* I ran grid search model for 1.5 days and pulled the plug because of time running thin. I would like to re-run my model post project completion to see if I can find an improvement in r^2 score.
    * Since the default model outperformed all random searches I'm interested to see if any tuning is needed, or if the default model truly scores the highest.

# 3. Model Visualizations -

## 3.1 Feature Importance - 

In [ ]:
#Set samp variable to show features when plotting 
samp = pd.DataFrame(XT_val,columns=ordinal_fit.get_feature_names())

In [ ]:
# Shap waterfall plot showing feature importance
explainer = shap.TreeExplainer(model_xgbr)
shap_values=explainer(samp.head(1))
shap.plots.waterfall(shap_values[0])

In [ ]:
# Shap force plot also showing feature importance
explainer = shap.TreeExplainer(model_xgbr)
shap_values = explainer.shap_values(samp.head(1))
shap.initjs()
shap.force_plot(base_value = explainer.expected_value,
                shap_values=shap_values,
                features=samp.head(1))

In [ ]:
# Permutation importance for features used in XGBR model
perm = PermutationImportance(model_xgbr,random_state=42).fit(XT_val,y_val)
eli5.show_weights(perm, feature_names = samp.columns.tolist())

## 3.2 Feature/Permutation Importance Conclusions - 

* Since shap doesn't seem to agree with RandomForestRegressor.. these are the results for the second best model.
* Generation biomass seems to have the highest importance, followed by generation nuclear.
* The feature engineered column seems to have the highest permutation importance value, followed by generation fossil gas.